In [11]:
import requests
import pandas as pd

def get_data():
    api_endpoint ='https://api.indeed.com/ads/apisearch'

    params = {
        'publisher': '448947293830904',
        'q': 'SDE',
        'l': 'ny',
        'jt': 'fulltime',
        'limit': 25,
        'start': 0 ,
        'radius': 100,
        'co': 'us',
        'userip': '1.2.3.4',
        'format': 'json',
        'useragent': 'Chrome',
        'v': 2
    }

    rows = []
    for i in range(40):
        response = requests.get(api_endpoint, params=params)
        rows.extend(response.json()['results'])

        params['start'] += 25

    tb = pd.DataFrame(rows)
    tb.to_csv('db.csv')

    

In [12]:
get_data()

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By

def get_description(page_url):
    
    
    # enter your codes here
    executable_path = '/opt/homebrew/bin/chromedriver'

    driver = webdriver.Chrome(executable_path=executable_path)

    driver.get(page_url)

    description = driver.find_element(By.CSS_SELECTOR, "div.jobsearch-jobDescriptionText")
    
    return description.text

In [8]:
db = pd.read_csv("db.csv",header=0,index_col=0)

db.drop(columns="indeedApply")
db.drop(columns="stations")
db.drop(columns="refNum")
db.drop(columns="onmousedown")
db.drop(columns="advertiserId")

description = []

for i in range(len(db)):
    description.append(get_description(db.url[i]))
db['description'] = description

db.to_csv('db_with_description.csv')

/var/folders/dc/wvln0gc568s99mg6ck45xlfc0000gn/T/ipykernel_17315/423266675.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=executable_path)


In [35]:
keywords = ['c', 'c++', 'c#', 'java', 'javascript', 'python', 'go', 'sql', 'nosql', 'redis', 'html', 'css', 'react', 'angular', 'vue', 'ruby', 'node.js', 'aws', 'django', '.net', 'redux', 'next.js', 'xml']

In [56]:
def tokenize(text):

    li = text.split()

    puncs = "!$%&'()*, -/:;<=>?@[\]^_`{|}~]"
    li2 = []

    for i in range(len(li)):
        li2.extend(li[i].split('/'))

    vocab = []
    for i in range(len(li2)):
        vocab.extend(li2[i].split('-'))
    
    for i in range(len(vocab)):
        vocab[i].strip()
        vocab[i] = ''.join(ch for ch in vocab[i] if ch not in puncs and ch != '"')
        vocab[i] = vocab[i].lower()
    
    return vocab

In [39]:
db = pd.read_csv("db_with_description.csv",header=0,index_col=0)
skills = []
for i in range(len(db)):
    vocab = tokenize(db.description[i])
    requires = []
    for word in vocab:
        if word in keywords:
            requires.append(word)
    requires = list(set(requires))
    skills.append(requires)
db['skills'] = skills
db.to_csv('db_with_skills.csv')


In [53]:
db = pd.read_csv("db_with_skills.csv",header=0,index_col=0)
titles = db['jobtitle']
processed_titles = []
for i in range(len(titles)):
    title = tokenize(titles[i])
    if 'back' in title or 'backend' in title:
        processed_titles.append('back end')
    elif 'front' in title or 'frontend' in title:
        processed_titles.append('front end')
    elif 'software' in title:
        processed_titles.append('software developer')
    else:
        if 'developer' in title:
            index = title.index('developer')
            processed_titles.append(' '.join(title[0:index + 1]))
        elif 'engineer' in title:
            index = title.index('engineer')
            processed_titles.append(' '.join(title[0:index + 1]))
        else:
            processed_titles.append(titles[i])
db['jobtitle'] = processed_titles
db.to_csv('final_db.csv')

In [52]:
import numpy as np
import ast

db = pd.read_csv("final_db.csv",header=0,index_col=0)

dtm = np.zeros((len(db), len(keywords)))
for i in range(len(db)):
    skills = ast.literal_eval(db.skills[i])
    for skill in skills:
        j = keywords.index(skill)
        dtm[i][j] = 1
total = dtm.sum(axis=0)
index = np.argsort(total)
word = []
for i in index:
    word.append(keywords[i])
value = total[index]
dic = dict(zip(word, value))
output = sorted(dic.items(), key = lambda item: item[1], reverse=True)
print(output)

[('java', 377.0), ('javascript', 335.0), ('python', 291.0), ('sql', 283.0), ('aws', 241.0), ('react', 198.0), ('css', 193.0), ('html', 161.0), ('c++', 136.0), ('angular', 119.0), ('c#', 119.0), ('.net', 92.0), ('c', 92.0), ('go', 89.0), ('nosql', 79.0), ('xml', 56.0), ('node.js', 54.0), ('ruby', 40.0), ('redux', 31.0), ('vue', 30.0), ('django', 26.0), ('redis', 26.0), ('next.js', 6.0)]


In [1]:
import PyPDF2

def read_pdf(file_name = 'resume.pdf'):
    with open(file_name, 'rb') as file:
        pdf_reader = PyPDF2.PdfFileReader(file)

        text = ""
        for page in range(pdf_reader.numPages):
            page_obj = pdf_reader.getPage(page)
            text += page_obj.extractText()
    return text


In [11]:

import spacy
import re
nlp = spacy.load("en_core_web_sm")

def preprocess(doc, lemmatized = True, pos_tag = False, remove_stopword = True, lower_case = True, remove_punctuation = True):
    docs = nlp(doc)
    sents = list(map(lambda s: s.text, docs.sents))
    tokenized_sents = list(map(lambda sent: preprocess_token(sent, lemmatized, pos_tag, remove_stopword, lower_case, remove_punctuation), docs.sents))
            
    return sents, tokenized_sents

def preprocess_token(sent, lemmatized, pos_tag, remove_stopword, lower_case, remove_punctuation):
    tokenized_sent = []
    for token in sent:
        text = token.text
        if lemmatized:
            text = token.lemma_

        if remove_stopword:
            if token.is_stop:
                continue

        if remove_punctuation:
            if token.is_punct:
                continue

        if bool(re.search("^\s*$", token.text)):
            continue

        if lower_case:
            text = text.lower()

        if pos_tag:
            text = (text, token.pos_)

        tokenized_sent.append(text)

    return tokenized_sent

In [12]:
sents, tokenized_sents = preprocess(read_pdf())

for i in range(3):
    print(sents[i], "\n",tokenized_sents[i],"\n\n" )

SHIHAO XIONG shihao.xiong@icloud.com   |  (551) 556 - 4100   |   Jersey City, NJ 07302  GitHub: https://github.com/sxiong5 | Linkedin: https://www.linkedin.com/in/sxiong5 SUMMARY      Seeking a Software Engineer. 
 ['shihao', 'xiong', 'shihao.xiong@icloud.com', '|', '551', '556', '4100', '|', 'jersey', 'city', 'nj', '07302', 'github', 'https://github.com/sxiong5', '|', 'linkedin', 'https://www.linkedin.com/in/sxiong5', 'summary', 'seek', 'software', 'engineer'] 


Master degree candidate. 
 ['master', 'degree', 'candidate'] 


A wide range of project experience in web programming. 
 ['wide', 'range', 'project', 'experience', 'web', 'programming'] 


